https://www.kaggle.com/karthikbhandary2/boat-sales -- вот датасет с яхтами.

1) Постройте модель зависимости цены яхты от ее параметров. 

2) Насколько модель точна? (в качестве метрики можно выбрать mean absolute error)

3) Какие факторы в какой степени влияют на цену?

P.S. Если вам покажется, что яхты это скучно, вы можете взять любой другой датасет с kaggle)

In [123]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
%matplotlib inline
import seaborn as sns
sns.set()

In [104]:
boat_data = pd.read_csv('boat_data.csv')

In [105]:
boat_data.head() # данные о цене представлены разными валютами, что не позволяет производить их сравнение. Необходиом все цены привести к одной валюте

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days
0,CHF 3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,Switzerland Â» Lake Geneva Â» VÃ©senaz,226
1,EUR 3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,Germany Â» BÃ¶nningstedt,75
2,CHF 3770,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,Switzerland Â» Lake of Zurich Â» StÃ¤fa ZH,124
3,DKK 25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,Denmark Â» Svendborg,64
4,EUR 3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,Germany Â» Bayern Â» MÃ¼nchen,58


In [106]:
boat_data.info() # имеются пропуски

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9888 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9888 non-null   object 
 1   Boat Type                    9888 non-null   object 
 2   Manufacturer                 8550 non-null   object 
 3   Type                         9882 non-null   object 
 4   Year Built                   9888 non-null   int64  
 5   Length                       9879 non-null   float64
 6   Width                        9832 non-null   float64
 7   Material                     8139 non-null   object 
 8   Location                     9852 non-null   object 
 9   Number of views last 7 days  9888 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 772.6+ KB


In [107]:
boat_data['Length'] = boat_data['Length'].fillna(boat_data['Length'].median()) # заполним числовые пропуски медианными значениями
boat_data['Length'] = boat_data['Length'].fillna(boat_data['Length'].median())

In [78]:
boat_data.info() # еще остались пропуски, их удалим 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9888 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9888 non-null   object 
 1   Boat Type                    9888 non-null   object 
 2   Manufacturer                 8550 non-null   object 
 3   Type                         9882 non-null   object 
 4   Year Built                   9888 non-null   int64  
 5   Length                       9888 non-null   float64
 6   Width                        9832 non-null   float64
 7   Material                     8139 non-null   object 
 8   Location                     9852 non-null   object 
 9   Number of views last 7 days  9888 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 772.6+ KB


In [108]:
boat_data = boat_data.dropna()

In [109]:
boat_data.info() # в годе выпуска есть нули, удалим их

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7019 entries, 1 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        7019 non-null   object 
 1   Boat Type                    7019 non-null   object 
 2   Manufacturer                 7019 non-null   object 
 3   Type                         7019 non-null   object 
 4   Year Built                   7019 non-null   int64  
 5   Length                       7019 non-null   float64
 6   Width                        7019 non-null   float64
 7   Material                     7019 non-null   object 
 8   Location                     7019 non-null   object 
 9   Number of views last 7 days  7019 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 603.2+ KB


In [115]:
boat_data = boat_data[boat_data['Year Built']!=0] # в годе выпуска есть нули, удалим их

In [116]:
def convert_into_usd(x):
    exchange_rate_based_usd = {"USD":1, "EUR":0.88,"CHF": 0.92, "DKK": 6.57,"Â£" : 0.75}
    l = x.split(" ")
    currency, amount = l[0], l[1]
    return np.round(float(amount) / exchange_rate_based_usd[currency], 2)
    
boat_data['price_in_USD'] = boat_data['Price'].apply(lambda x: convert_into_usd(x))

C:\Users\user\AppData\Local\Temp/ipykernel_15028/2457573260.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat_data['price_in_USD'] = boat_data['Price'].apply(lambda x: convert_into_usd(x))


1) Постройте модель зависимости цены яхты от ее параметров

Для определения зависимости цены от ее параметров можно 

In [117]:
boat_data   # получили данные о цене в долларах

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days,price_in_USD
1,EUR 3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,Germany Â» BÃ¶nningstedt,75,3965.91
4,EUR 3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,Germany Â» Bayern Â» MÃ¼nchen,58,3862.50
10,CHF 3500,Fishing Boat,Terhi power boats,"Used boat,Electric",1987,4.35,1.75,GRP,Switzerland Â» Seengen,239,3804.35
12,EUR 3500,Sport Boat,GS Nautica power boats,Used boat,2004,4.70,2.00,GRP,Italy Â» Lake Garda Â» Moniga del Garda (BS),69,3977.27
13,CHF 4600,Runabout,Kimple power boats,new boat from stock,2020,4.40,1.65,Aluminium,Switzerland Â» Zugersee Â» Neuheim,113,5000.00
...,...,...,...,...,...,...,...,...,...,...,...
9877,CHF 5000,Cabin Boat,Princess power boats,Used boat,1974,7.20,2.70,GRP,Switzerland Â» Rhein Â» MÃ¶hlin,1299,5434.78
9878,CHF 4999,"Sport Boat,Wakeboard/Wakesurf,Water ski",Tullio Abbate power boats,"Used boat,Unleaded",1980,6.00,2.10,GRP,Switzerland Â» Lake of Zurich Â» Rafz,1917,5433.70
9882,CHF 4950,Fishing Boat,Staempfli power boats,"Used boat,Unleaded",1984,6.00,1.62,Plastic,Switzerland Â» Bielersee Â» Gerolfingen,288,5380.43
9885,EUR 4499,Sport Boat,BlueCraft power boats,"new boat from stock,Unleaded",2020,4.40,1.80,GRP,Germany Â» Nordrhein-Westfalen Â» Wesel,354,5112.50


Построим модели для предсказания, поделим выборку, выберем переменные и трансформируем качественные переменные в бинарные 

In [141]:
X = boat_data[['Boat Type', 'Type', 'Material', 'Number of views last 7 days']]
y = boat_data['price_in_USD']

In [142]:
X = pd.get_dummies(X)

In [143]:
X.head()

,Number of views last 7 days,Boat Type_Bowrider,"Boat Type_Bowrider,Cabin Boat,Deck Boat","Boat Type_Bowrider,Center console boat,Sport Boat","Boat Type_Bowrider,Classic","Boat Type_Bowrider,Deck Boat,Water ski","Boat Type_Bowrider,Motor Yacht,Sport Boat","Boat Type_Bowrider,Motor Yacht,Wakeboard/Wakesurf","Boat Type_Bowrider,Sport Boat,Wakeboard/Wakesurf","Boat Type_Bowrider,Wakeboard/Wakesurf",...,Material_Aluminium,Material_Carbon Fiber,Material_GRP,Material_Hypalon,Material_PVC,Material_Plastic,Material_Rubber,Material_Steel,Material_Thermoplastic,Material_Wood
1,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,58,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10,239,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
12,69,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
13,113,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [144]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [145]:
X_train.shape   # объем тренировочной 

(4941, 143)

In [146]:
X_test.shape  # объем тестовой

(1648, 143)

In [158]:
import sklearn
from sklearn.linear_model import LogisticRegression

In [159]:
lg = LogisticRegression()
lg.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [151]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4941 entries, 7494 to 9378
Columns: 143 entries, Number of views last 7 days to Material_Wood
dtypes: int64(1), uint8(142)
memory usage: 762.4 KB


In [ ]:
lg_prediction = lg.predict(X_test[X_train.columns])
rf_prediction = rf.predict(X_test[X_train.columns])

In [ ]:
from sklearn.metrics import accuracy_score

print("Log regression accuracy:", accuracy_score(y_test, lg_prediction))

In [ ]:
list(zip(X_train.columns, lg.coef_[0]))